In [ ]:
# T bounds 324-524 K
# P bounds 0.1-36 atm
# Steady-state operation with ideal mixing, ideal phase equilibrium, and ideal gas behaviour.
import numpy as np
from scipy.optimize import fsolve
# stream and process unit tracker class
class Streams():
    _instances = []
    def __init__(self,name):
        self.__class__._instances.append(self)
        self.name = name
    @classmethod
    def get_all(cls):
        return cls._instances

    @classmethod
    def clear_registry(cls):
        cls._instances = []


class Stream(Streams):
    def __init__(self, name, T, P, F,z=[0,0,0,0]):
        # 1 Ethylene Oxide
        # 2 Water
        # 3 Ethylene Glycol
        # 4 Diethylene Glycol
        super().__init__(name)
        self.T = T
        self.P = P
        self.F = F
        self.z = np.array(z)
        self.x = z
        self.y = np.zeros(4)
    def __str__(self):
        return f"Stream {self.name}: T={self.T} K, P={self.P} atm, F={self.F} kmol/h, z={self.z}, x={self.x}, y={self.y}"

    def saturation_pressure(self):
        sat_data = np.array([
            [10.884,3152,7.667],
            [11.680,3828,-45.412],
            [11.963,4764,-72.275],
            [11.256,4655,103.551],
            ])
        psats= np.exp(sat_data[:,0] - sat_data[:,1]/(sat_data[:,2]+self.T))
        return psats
        
    def liquid_molar_density(self):
        #kmol/m3
        lmdp = np.array([
            [1.655, 0.126300,-0.0002286],
            [75.22, -0.065050, 0],
            [23.66, -0.018040, 0],
            [13.59, -0.009986, 0],
            ])
        
        liquid_molar_densities = lmdp[:,0] + lmdp[:,1]*self.T + lmdp[:,2]*self.T**2
        return liquid_molar_densities

    def H(self):
        hl_data = np.array([
            [-143.8, 0.2037],
            [-320, 0.1068],
            [-519, 0.1924],
            [-723, 0.3043],
            ])

        hv_data = np.array([
            [-74.6, 0.0697],
            [-252.3, 0.0348],
            [-424.3, 0.1030],
            [-603.4, 0.1823],
            ])
        
        hls = hl_data[:,0] + hl_data[:,1]*self.T
        hvs = hv_data[:,0] + hv_data[:,1]*self.T
        
        hl = sum(hls*self.x)
        hv = sum(hvs*self.y)
        #data looks like it is in j/kmol, convert to kJ/kmol
        return self.F*hl*1000

class ProcessUnits():
    _instances = []
    def __init__(self, name):
        self.__class__._instances.append(self)
        self.name = name

    @classmethod
    def get_all(cls):
        return cls._instances.copy()

    @classmethod
    def clear_registry(cls):
        cls._instances = []
        
class Heater(ProcessUnits):
    def __init__(self, name, inlet_stream, Tout):
        super().__init__(name)
        if inlet_stream.T > Tout:
            raise ValueError("Heater inlet temperature must be less than outlet temperature.")
        self.Tout = Tout
        self.outlet= Stream(name+"_outlet", Tout, inlet_stream.P, inlet_stream.F, inlet_stream.z)
        self.Q = self.outlet.H() - inlet_stream.H()

class Cooler(ProcessUnits):
    def __init__(self, name, inlet_stream, Tout):
        super().__init__(name)
        if inlet_stream.T < Tout:
            raise ValueError("Cooler inlet temperature must be greater than outlet temperature.")
        self.Tout = Tout
        self.outlet= Stream(name+"_outlet", Tout, inlet_stream.P, inlet_stream.F, inlet_stream.z)
        self.Q = inlet_stream.H() - self.outlet.H()

class Mixer(ProcessUnits):
    def __init__(self, name, inlet_stream1, inlet_stream2, inlet_stream3=None, inlet_stream4=None):
        super().__init__(name)
        self.inlet_stream1 = inlet_stream1
        self.inlet_stream2 = inlet_stream2
        self.inlet_stream3 = inlet_stream3
        self.inlet_stream4 = inlet_stream4
        P = min(inlet_stream1.P, inlet_stream2.P, (inlet_stream3.P if inlet_stream3 else float('inf')), (inlet_stream4.P if inlet_stream4 else float('inf')))
        F = inlet_stream1.F + inlet_stream2.F + (inlet_stream3.F if inlet_stream3 else 0) + (inlet_stream4.F if inlet_stream4 else 0)
        T = (inlet_stream1.F*inlet_stream1.T + inlet_stream2.F*inlet_stream2.T + (inlet_stream3.F*inlet_stream3.T if inlet_stream3 else 0) + (inlet_stream4.F*inlet_stream4.T if inlet_stream4 else 0)) / F
        z = (inlet_stream1.F*inlet_stream1.z + inlet_stream2.F*inlet_stream2.z + (inlet_stream3.F*inlet_stream3.z if inlet_stream3 else 0) + (inlet_stream4.F*inlet_stream4.z if inlet_stream4 else 0)) / F
        self.outlet = Stream(name+"_outlet", T, P, F, z)

class CSTReactor(ProcessUnits):
    def __init__(self, name, inlet_stream,volume=0):
        super().__init__(name)
        self.inlet_stream = inlet_stream
        self.volume = volume
    def reaction_rates(self, T):
        """Calculate reaction rates r1 and r2 at given temperature T."""
        k1 = 13706.91 * np.exp(-8220 / T)
        k2 = 96341.59 * np.exp(-8700 / T)
        return k1, k2
    nu = np.array([[-1, -1, 1, 0], [-1, 0, -1, 1]])
    
    def H(T):
        # Enthalpy data [h0, cp] (kJ/kmol, kJ/kmol-K)
        hl_data = np.array([
        [-143.8, 0.2037],  
        [-320.0, 0.1068],  
        [-519.0, 0.1924],  
        [-723.0, 0.3043],
        ])
        
        """Calculate component enthalpies at temperature T (K)"""
        return (hl_data[:, 0] + hl_data[:, 1] * (T - 298.15))*1000  # kJ/kmol

    def rate(C, T):
        """Calculate reaction rates (kmol/m³·hr)"""
        C_A, C_B, C_C, C_D = C
        k1 = 13706.91 * np.exp(-8220 / T) * 3600  # hr⁻¹
        r1 = k1 * C_A * C_B  # kmol/m³·hr
        k2 = 96341.59 * np.exp(-8700 / T) * 3600  # hr⁻¹
        r2 = k2 * C_A * C_C  # kmol/m³·hr
        return r1, r2

    h_A0, h_B0, h_C0, h_D0 = H(T0)

    def liquid_molar_density(T):
        """Calculate pure component liquid molar densities (kmol/m³)"""
        lmdp = np.array([
            [1.655, 0.126300, -0.0002286],  # A
            [75.22, -0.065050, 0],          # B
            [23.66, -0.018040, 0],          # C
            [13.59, -0.009986, 0],          # D
        ])
        return lmdp[:, 0] + lmdp[:, 1] * T + lmdp[:, 2] * T**2

    def mixture_density(C, T):
        """Calculate mixture density using partial molar volumes"""
        rho_pure = liquid_molar_density(T)
        x = C / np.sum(C)  # Mole fractions
        # Assuming ideal mixing (partial molar volume = pure component molar volume)
        v_mix = np.sum(x / rho_pure)  # m³/kmol
        return 1 / v_mix  # kmol/m³

    def equations(vars):
        C_A, C_B, C_C, C_D, T = vars
        
        # Check for physical temperature bounds
        if T < 300 or T > 500:
            return [1e6]*5  # Large residual to guide solver back
        if any(c < 0 for c in [C_A, C_B, C_C, C_D]):
            return [1e6]*5
        
        # Get mixture density (kmol/m³)
        C_tot = C_A + C_B + C_C + C_D
        rho_tot = mixture_density([C_A, C_B, C_C, C_D], T)
        
        # Outlet flow rate (m³/hr) - mass balance
        Q_out = Q_flow * rho_0 / rho_tot
        
        # Reaction rates (kmol/m³·hr)
        r1, r2 = rate([C_A, C_B, C_C, C_D], T)
        
        # Mass balances (kmol/hr)
        resid_A = C_A0 * Q_flow - C_A * Q_out + V * (nu[0, 0] * r1 + nu[1, 0] * r2)
        resid_B = C_B0 * Q_flow - C_B * Q_out + V * (nu[0, 1] * r1 + nu[1, 1] * r2)
        resid_C = C_C0 * Q_flow - C_C * Q_out + V * (nu[0, 2] * r1 + nu[1, 2] * r2)
        resid_D = C_D0 * Q_flow - C_D * Q_out + V * (nu[0, 3] * r1 + nu[1, 3] * r2)
        
        # Heat of reactions (kJ/kmol) - negative for exothermic reactions
        delta_H_rxn1 = -(-0.1181 * T - 55.2) *1000 # kJ/kmol
        delta_H_rxn2 = -(-0.0918 * T - 60.2) *1000 # kJ/kmol
        
        # Enthalpy terms (kJ/hr)
        h_A, h_B, h_C, h_D = H(T)
        hin = (C_A0 * h_A0 + C_B0 * h_B0 + C_C0 * h_C0 + C_D0 * h_D0) * Q_flow
        hout = (C_A * h_A + C_B * h_B + C_C * h_C + C_D * h_D) * Q_out
        
        # Energy balance residual (kJ/hr)
        Q_rxn = (delta_H_rxn1 * r1 + delta_H_rxn2 * r2) * V  # kJ/hr
        resid_T = (hin - hout) + Q_rxn

        return [resid_A, resid_B, resid_C, resid_D, resid_T]

    # Solve with better initial guess
    guess = [2.17, 44.85, 0.02, 0.0, 359.27]  # Better than feed conditions
    solution = fsolve(equations, guess, xtol=1e-6)
    

In [ ]:
# T Kelvin, P atm, n kmol/h, z1 mol fraction of Ethylene oxide, z2 mol fraction of water
# Feed1 = Stream('Feed1', 298,2.4,26.32,z=[0,1,0,0])
# Feed2 = Stream('Feed2', 298,2.4,27.62,z=[1,0,0,0])
# D1 = Stream('D1', 360.5,2.4,522.37,z=[0.00164634263,0.99787506939,0.00047858798,0])
# Mixer1 = Mixer('Mixer1', Feed1, Feed2, D1)
# Heater1 = Heater('Heater1', Mixer1.outlet, 355)
# print(Heater1.outlet)
# print(Heater1.outlet.liquid_molar_density())
# print(Heater1.outlet.F*Heater1.outlet.z)
# v = np.sum(Heater1.outlet.F*Heater1.outlet.z/Heater1.outlet.liquid_molar_density())
# print(f"Volumetric flow: {v} m3/hr")
# print(f"Molar flow: {Heater1.outlet.F} kmol/hr")
# print(Heater1.outlet.F*Heater1.outlet.z/v)
CSTRInlet = Stream('CSTRInlet', 355, 2.4, 576.317942978094, z=[0.0494255625707587,0.950134575786539,0.000439861459002414,1.83699843408081E-10])
Q = np.sum(CSTRInlet.F*CSTRInlet.z/CSTRInlet.liquid_molar_density())
print(f"Volumetric flow: {Q} m3/hr")
print(f"Molar flow: {CSTRInlet.F} kmol/hr")
print(CSTRInlet.F/Q)
print(CSTRInlet.z)
print(1/np.sum(CSTRInlet.z/CSTRInlet.liquid_molar_density()))
print(CSTRInlet.H())

In [ ]:
#heat of reaction calculation
#R1
dhr1 = []
dhr2 = []
for T in range(298, 525):
    reactants = Stream("reactants", T,1,2,[0.5,0.5,0,0])
    products = Stream("products", T,1,1,[0,0,1,0])
    heat_of_reaction = products.H() - reactants.H()
    dhr1.append(heat_of_reaction)
    #R2
    reactants2 = Stream("reactants2", T,1,2,[0.5,0,0.5,0])
    products2 = Stream("products2", T,1,1,[0,0,0,1])
    heat_of_reaction2 = products2.H() - reactants2.H()
    dhr2.append(heat_of_reaction2)
dhr1 = np.array(dhr1)
dhr2 = np.array(dhr2)
m,b = np.polyfit(range(298, 525), dhr1, 1)
print(f"R1: dH = {m}*T + {b}")
m,b = np.polyfit(range(298, 525), dhr2, 1)
print(f"R2: dH = {m}*T + {b}")

In [ ]:
flow_data_array = np.array([
    [0.00, 355.0, 547.58, 28.48, 0.25, 0.00],
    [3.75, 359.3, 545.49, 26.38, 2.33, 0.02],
    [7.50, 365.0, 542.85, 23.69, 4.93, 0.06],
    [11.25, 372.6, 539.35, 20.09, 8.32, 0.17],
    [15.00, 382.7, 534.70, 15.22, 12.75, 0.39],
    [18.75, 394.4, 529.35, 9.50, 17.72, 0.76],
    [22.50, 403.8, 525.05, 4.81, 21.65, 1.14],
    [26.25, 409.3, 522.59, 2.11, 23.86, 1.38],
    [30.00, 411.8, 521.46, 0.87, 24.87, 1.50]
])
#difference between stepsp
print("DV, DT, DW, DEO, DEG, DDEG")
print(flow_data_array[1,:]-flow_data_array[0,:])
print(flow_data_array[2,:]-flow_data_array[1,:])
print(flow_data_array[3,:]-flow_data_array[2,:])

In [ ]:
import numpy as np
from scipy.optimize import fsolve

# Reactor parameters (units: kmol/m³, kJ/kmol, K)
V = 3.75  # m³
Q_flow = 12.130295903042116  # m³/hr
F = 576.317942978094  # kmol/hr
C_A0 = 2.34823938e+00  # kmol/m³
C_B0 = 4.51414878e+01  # kmol/m³
C_C0 = 2.08980929e-02  # kmol/m³
C_D0 = 8.72769442e-09  # kmol/m³
T0 = 355.0  # K
rho_0 = 47.510625262946895  # kmol/m³ (inlet density)

# Stoichiometric coefficients
nu = np.array([[-1, -1, 1, 0], [-1, 0, -1, 1]])

# Enthalpy data [h0, cp] (kJ/kmol, kJ/kmol-K)
hl_data = np.array([
    [-143.8, 0.2037],  
    [-320.0, 0.1068],  
    [-519.0, 0.1924],  
    [-723.0, 0.3043],  
])

def H(T):
    """Calculate component enthalpies at temperature T (K)"""
    return (hl_data[:, 0] + hl_data[:, 1] * (T-298))*1000  # kJ/kmol

def rate(C, T):
    """Calculate reaction rates (kmol/m³·hr)"""
    C_A, C_B, C_C, C_D = C
    k1 = 13706.91 * np.exp(-8220 / T) * 3600  # hr⁻¹
    r1 = k1 * C_A * C_B  # kmol/m³·hr
    k2 = 96341.59 * np.exp(-8700 / T) * 3600  # hr⁻¹
    r2 = k2 * C_A * C_C  # kmol/m³·hr
    return r1, r2

h_A0, h_B0, h_C0, h_D0 = H(T0)

def liquid_molar_density(T):
    """Calculate pure component liquid molar densities (kmol/m³)"""
    lmdp = np.array([
        [1.655, 0.126300, -0.0002286],  # A
        [75.22, -0.065050, 0],          # B
        [23.66, -0.018040, 0],          # C
        [13.59, -0.009986, 0],          # D
    ])
    return lmdp[:, 0] + lmdp[:, 1] * T + lmdp[:, 2] * T**2

def mixture_density(C, T):
    """Calculate mixture density using partial molar volumes"""
    rho_pure = liquid_molar_density(T)
    x = C / np.sum(C)  # Mole fractions
    # Assuming ideal mixing (partial molar volume = pure component molar volume)
    v_mix = np.sum(x / rho_pure)  # m³/kmol
    return 1 / v_mix  # kmol/m³

def equations(vars):
    C_A, C_B, C_C, C_D, T = vars
    
    # Check for physical temperature bounds
    if T < 300 or T > 500:
        return [1e6]*5  # Large residual to guide solver back
    if any(c < 0 for c in [C_A, C_B, C_C, C_D]):
        return [1e6]*5
    
    # Get mixture density (kmol/m³)
    C_tot = C_A + C_B + C_C + C_D
    rho_tot = mixture_density([C_A, C_B, C_C, C_D], T)
    
    # Outlet flow rate (m³/hr) - mass balance
    Q_out = Q_flow * rho_0 / rho_tot
    
    # Reaction rates (kmol/m³·hr)
    r1, r2 = rate([C_A, C_B, C_C, C_D], T)
    
    # Mass balances (kmol/hr)
    resid_A = C_A0 * Q_flow - C_A * Q_out + V * (nu[0, 0] * r1 + nu[1, 0] * r2)
    resid_B = C_B0 * Q_flow - C_B * Q_out + V * (nu[0, 1] * r1 + nu[1, 1] * r2)
    resid_C = C_C0 * Q_flow - C_C * Q_out + V * (nu[0, 2] * r1 + nu[1, 2] * r2)
    resid_D = C_D0 * Q_flow - C_D * Q_out + V * (nu[0, 3] * r1 + nu[1, 3] * r2)
    
    # Heat of reactions (kJ/kmol) - negative for exothermic reactions
    delta_H_rxn1 = -(-0.1181 * T - 55.2) *1000 # kJ/kmol
    delta_H_rxn2 = -(-0.0918 * T - 60.2) *1000 # kJ/kmol
    
    # Enthalpy terms (kJ/hr)
    h_A, h_B, h_C, h_D = H(T)
    hin = (C_A0 * h_A0 + C_B0 * h_B0 + C_C0 * h_C0 + C_D0 * h_D0) * Q_flow
    hout = (C_A * h_A + C_B * h_B + C_C * h_C + C_D * h_D) * Q_out
    
    # Energy balance residual (kJ/hr)
    Q_rxn = (delta_H_rxn1 * r1 + delta_H_rxn2 * r2) * V  # kJ/hr
    # Q_rxn = 0
    resid_T = (hin - hout) + Q_rxn

    return [resid_A, resid_B, resid_C, resid_D, resid_T]

# Solve with better initial guess
guess = [2.17, 44.85, 0.02, 0.0, 359.27]  # Better than feed conditions
solution = fsolve(equations, guess, xtol=1e-6)

# Unpack solution
C_A_sol, C_B_sol, C_C_sol, C_D_sol, T_sol = solution

# Calculate outlet flow rates
rho_tot_out = mixture_density([C_A_sol, C_B_sol, C_C_sol, C_D_sol], T_sol)
Q_out = Q_flow * rho_0 / rho_tot_out  # Outlet volumetric flow (m³/hr)
F_A_out = C_A_sol * Q_out  # kmol/hr
F_B_out = C_B_sol * Q_out  # kmol/hr
F_C_out = C_C_sol * Q_out  # kmol/hr
F_D_out = C_D_sol * Q_out  # kmol/hr
F_total_out = F_A_out + F_B_out + F_C_out + F_D_out  # kmol/hr

# Print results
print("\nAdiabatic CSTR Results (Variable Density):")
print(f"Temperature: {T_sol:.2f} K (Inlet: {T0:.2f})")
print(f"Outlet density: {rho_tot_out:.3f} kmol/m³ (Inlet: {rho_0:.3f})")
print(f"Outlet volumetric flow: {Q_out:.3f} m³/hr (Inlet: {Q_flow:.3f})")

print("\nMolar Flow Rates (kmol/hr):")
print(f"{'Component':<10}{'Inlet':>10}{'Outlet':>10}{'Change':>10}")
print(f"{'A':<10}{C_A0*Q_flow:>10.3f}{F_A_out:>10.3f}{F_A_out-C_A0*Q_flow:>10.3f}")
print(f"{'B':<10}{C_B0*Q_flow:>10.3f}{F_B_out:>10.3f}{F_B_out-C_B0*Q_flow:>10.3f}")
print(f"{'C':<10}{C_C0*Q_flow:>10.3f}{F_C_out:>10.3f}{F_C_out-C_C0*Q_flow:>10.3f}")
print(f"{'D':<10}{C_D0*Q_flow:>10.3f}{F_D_out:>10.3f}{F_D_out-C_D0*Q_flow:>10.3f}")
print(f"{'Total':<10}{F:>10.3f}{F_total_out:>10.3f}{F_total_out-F:>10.3f}")